# Demo: Grid model and calculate capacity headroom

In [1]:
import pandas as pd
import numpy as np
import copy
import pandapower as pp
import yaml
import folium
import copy
import plotly.graph_objs as go
import json
import pandapower.networks as pn
from datetime import date, datetime
import os,sys,inspect

#To import packages from parent subfolder 
currentdir =  os.path.abspath(os.getcwd())
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)


from capacitymap.grid.grid import Grid, generate_config_from_project, combine_dict, daterange
from capacitymap.analysis import analysis_check
from capacitymap.analysis import capacity_analysis
analysis_check.check_violations.counter = 0


## Pandapower grid model with shunt and tap controllers

In [2]:
# Pandapower grid
net = pp.from_json(os.path.join(currentdir,'data\\svedala\\svedala.json')) 

In [3]:
#without controllers
pp.runpp(net, run_control=False)
voltage_violated_bus = pp.violated_buses(net, min_vm_pu=0.9, max_vm_pu=1.1)
net.res_bus.loc[voltage_violated_bus]

,vm_pu,va_degree,p_mw,q_mvar
21,0.820328,-66.851904,6.0,0.500000
211,0.894834,-67.899519,1.5,0.380000
221,0.882192,-68.322718,8.0,1.500000
231,0.807027,-66.580029,8.0,1.500000
261,0.875782,-62.694181,1.5,0.380000
292,0.864653,-57.997382,800.0,461.247845
312,0.806004,-75.010674,600.0,129.040218
323,1.117040,3.309774,-400.0,-122.812440
371,0.888407,-66.275367,8.0,1.500000
441,0.862078,-61.266689,700.0,409.288912


In [4]:
net.controller.loc[5].object.__dict__

{'matching_params': {},
 'index': 5,
 'sid': 5,
 'bid': 151,
 'element_in_service': 'True',
 'controlled_bus': 151,
 'tap_min': 0,
 'tap_max': 4,
 'init_step': 4,
 'tap_pos': 4,
 'tol': 4,
 'vm_lower_pu': 1.00312,
 'vm_upper_pu': 1.03437,
 'steps': [400.0, 0, -150.0, -300.0, -400.0],
 'tested_steps': []}

In [5]:
#without controllers
pp.runpp(net, run_control=True)
voltage_violated_bus = pp.violated_buses(net, min_vm_pu=0.9, max_vm_pu=1.1)
net.res_bus.loc[voltage_violated_bus]

,vm_pu,va_degree,p_mw,q_mvar
284,1.10000,-4.878401,-300.0,-79.600471
323,1.11704,3.971423,-400.0,-167.166872


In [6]:
net.controller.loc[5].object.__dict__

{'matching_params': {},
 'index': 5,
 'sid': 5,
 'bid': 151,
 'element_in_service': 'True',
 'controlled_bus': 151,
 'tap_min': 0,
 'tap_max': 4,
 'init_step': 4,
 'tap_pos': 0,
 'tol': 4,
 'vm_lower_pu': 1.00312,
 'vm_upper_pu': 1.03437,
 'steps': [400.0, 0, -150.0, -300.0, -400.0],
 'tested_steps': [3, 2, 1, 0]}

## Grid Capacity datamodel

In [8]:
# Pandapower grid
net = pp.from_json(os.path.join(currentdir,'data\\svedala\\svedala.json')) 

# Dataframe over planned projects, to start this list is empty
projects = pd.DataFrame(columns=['Start', 'End', 'Object_type','ObjectID', 'Status'])

# Create grid object
svedala = Grid('Svedala',net, {}, projects)


In [9]:
go.Figure(svedala.plot_traces())

## Define limits and contingency test
Before we can start with the capacity calculation we need to define the thresholds for the test we check for every power flow we run. The analysis builds on 8 test, if one of these fails the enterie scenario fails. The tests are:
1. Upper limit on bus bus voltage
2. Lower limit on bus voltage
3. Max loading on line exceeded
4. Max loading on trafo exceeded
5. Subscription limit to overlying network exceeded
6. Unsupplied loads
7. Power flow did not converge, meaning something is really wrong :)

Dependning on what test we run these thresholds can vary. For capacity calculation we run two types of calculations, one for normal operations and one for contingency scenarios (relevant N-1). We therfore define two sets of limits to validate available capacity:

In [10]:
# Normal operation limits
normal_limits = {'vmin': 0.9,
                'vmax': 1.12,
                'max_line_loading' : 105,
                'max_trafo_loading' : 100,
                'subscription_p_limits': 1000,
                'run_controllers': True}

In [11]:
analysis_check.check_violations(svedala.grid,vmax=normal_limits['vmax'], 
                                vmin=normal_limits['vmin'], 
                                max_line_loading=normal_limits['max_line_loading'], 
                                max_trafo_loading=normal_limits['max_trafo_loading'],
                                p_lim=normal_limits['subscription_p_limits'],
                                run_control=normal_limits['run_controllers'])

((False, False, False, False, False, False, False),
 [None, None, None, None, None, None, None])

In [12]:
analysis_check.check_violations.counter

1

For the contingency test, we need to define which objects to test. The list consists of two lists, one over lines to test and one over trafos to test.

In [13]:

fileObject = open(os.path.join(currentdir,"data\\svedala\\line_to_test.json"), "r")
jsonContent = fileObject.read()
lines_to_test = json.loads(jsonContent)

fileObject = open(os.path.join(currentdir,"data\\svedala\\trafo_to_test.json"), "r")
jsonContent = fileObject.read()
trafos_to_test = json.loads(jsonContent)

In [14]:
contingency_scenario = [lines_to_test, trafos_to_test]


In [15]:
# Limits for continency tests
contingency_limits = {'vmin': 0.87,
                'vmax': 1.15,
                'max_line_loading' : 105,
                'max_trafo_loading' : 100,
                'subscription_p_limits': 1000,
                'run_controllers': True}
cl,ct =analysis_check.contingency_test(net,vmax=contingency_limits['vmax'],
                           vmin=contingency_limits['vmin'],
                           max_line_loading=contingency_limits['max_line_loading'],
                           max_trafo_loading=contingency_limits['max_trafo_loading'],
                           p_lim=contingency_limits['subscription_p_limits'],
                           run_control=contingency_limits['run_controllers'],
                           contingency_scenario=contingency_scenario)

In [16]:
print('Critical lines: ')
print(cl)
print('Critical trafos: ')
print(ct)

Critical lines: 
[]
Critical trafos: 
[]


## Define capacity headroom (small grid 57 bus)

In [17]:
# Pandapower grid
net = pn.case57()#create_cigre_network_hv(length_km_6a_6b=0.1)
net.bus.name = ['Bus '+str(x) for x in net.bus.index]
#net.shunt.q_mvar = net.shunt.q_mvar*1000

# Dataframe over planned projects, to start this list is empty
projects = pd.DataFrame(columns=['Start', 'End', 'Object_type','ObjectID', 'Status'])

# Create grid object
grid = Grid('small test grid',net, {}, projects)

In [18]:
net

This pandapower network includes the following parameter tables:
   - bus (57 elements)
   - load (42 elements)
   - gen (6 elements)
   - shunt (3 elements)
   - ext_grid (1 element)
   - line (63 elements)
   - trafo (17 elements)
   - poly_cost (7 elements)
   - bus_geodata (57 elements)

In [19]:
# Define type of capacity (generation or load)
cap_type = 'load'

# Define max headroom to test
upper_lim_p = 500

## Calculate capacity headroom

In [20]:
#Limits
normal_limits = {'vmin': 0.90,
                'vmax': 1.10,
                'max_line_loading' : 105,
                'max_trafo_loading' : 105,
                'subscription_p_limits': 1000,
                'run_controllers': True}

contingency_limits = {'vmin': 0.85,
                'vmax': 1.15,
                'max_line_loading' : 120,
                'max_trafo_loading' : 120,
                'subscription_p_limits': 1000,
                'run_controllers': True}
cap_headroom = capacity_analysis.headroom(grid.grid, cap_type,upper_lim_p, 
                                          normal_limits=normal_limits, 
                                          contingency_limits=contingency_limits,
                                          contingency_scenario=[[],[]])

Max capacity is available
[====================] 100% (number of PFs: 449)

In [21]:
go.Figure(grid.plot_traces(plot_capacity=True, cap_res=cap_headroom))